In [1]:
from openpyxl import load_workbook
import pandas as pd
from kopyt import Parser, node

In [2]:
!pip install -U kopyt

  Using cached kopyt-0.0.2-py3-none-any.whl (33 kB)


In [3]:
df = pd.read_excel('FinalDataset-test0.xlsx', sheet_name = 'data' )

In [4]:
print(df.columns)

Index(['package', 'xml', 'png', 'resourceId', 'filePath', 'location'], dtype='object')


In [5]:
def getfileContent(path):
    f = open(path, "r")
    content = f.read()
    
    return content

In [6]:
def nested(result, line):
    
    
    finalCode = ""
    for declaration in result: 
        # print(type(declaration))
        if( isinstance(declaration, node.FunctionDeclaration) and line > declaration.position.line):
            finalCode = str(declaration)
        
        elif( isinstance(declaration, node.PropertyDeclaration) and line == declaration.position.line):
            return finalCode
        
        elif( isinstance(declaration, node.EnumDeclaration) and line > declaration.position.line):
            finalCode = str(declaration)
        
        elif( isinstance(declaration, node.AnonymousInitializer) and line > declaration.position.line):
            finalCode = str(declaration)
        
        elif( isinstance(declaration, node.ClassDeclaration) and line >declaration.position.line and declaration.body is not None):
            finalCode = nested( declaration.body.members , line)
            
        elif( isinstance(declaration, node.CompanionObject) and line > declaration.position.line and declaration.body is not None):
            finalCode = nested( declaration.body.members , line)
    
    return finalCode


def getResultForKotlin(code, line):
    
    
    parser = Parser(code)
    result = parser.parse()
    
    finalCode = ""

    # print(type(result), type(result.declarations))
    for declaration in result.declarations: 
        
        # print(type(declaration))
        if( isinstance(declaration, node.FunctionDeclaration) and line > declaration.position.line):
            finalCode = str(declaration)
        
        elif( isinstance(declaration, node.PropertyDeclaration) and line == declaration.position.line):
            return finalCode
        
        elif( isinstance(declaration, node.AnonymousInitializer) and line > declaration.position.line):
            finalCode = str(declaration)
            
        elif( isinstance(declaration, node.EnumDeclaration) and line > declaration.position.line):
            finalCode = str(declaration)
            
        elif( isinstance(declaration, node.ClassDeclaration) and line > declaration.position.line and declaration.body is not None):
            finalCode = nested( declaration.body.members , line)
            
        elif( isinstance(declaration, node.CompanionObject) and line > declaration.position.line and declaration.body is not None):
            # finalCode = str(declaration)
            finalCode = nested( declaration.body.members , line)
                
    return finalCode

In [7]:
def formatResourceId(resourceId: str):
    words = resourceId.split('_')
    result: str = ""
    for i in enumerate(words):
        if( i[0] == 0 ):
            result = result + i[1]
        else:
            result = result + i[1].capitalize()
    
    return result

In [6]:
def filterOverlappingName( data, column, resourceId):
    column = column - 1
    
    # print(len(data), column)
    if( column == 0 ):
        return True
    if data[column-1].isalpha() == True:
        return False
    if data[column-1] == '_':
        return False
    
    
    if len(data) <= column + len(resourceId):
        return True;
    if data[column + len(resourceId)].isalpha() == True:
        return False
    if data[column + len(resourceId)] == '_':
        return False
    
    return True
    

In [7]:
print(filterOverlappingName("previous", 1, "previous"))
print(filterOverlappingName("  previous", 3, "previous"))
print(filterOverlappingName("  MyPrevious", 5, "previous"))
print(filterOverlappingName("  M.Previous", 5, "previous"))
print(filterOverlappingName(".previous  ", 2, "previous"))
print(filterOverlappingName("previous  ", 1, "previous"))
print(filterOverlappingName(".previousDay", 2, "previous"))
print(filterOverlappingName(".previous ", 2, "previous"))
print(filterOverlappingName(".previous.m", 2, "previous"))
print(filterOverlappingName("l.previous", 3, "previous"))

True
True
False
True
True
True
False
True
True
True


In [10]:
result = []
counter = 0
failed = 0

for index, row in df.iterrows():
    
    path = row['filePath']
    line = int( row['location'].split(',')[0].split('=')[-1])
    col = int(row['location'].split(',')[1].split('=')[-1])
    resourceId = formatResourceId(row['resourceId'].split('/')[-1])
    
    
    code = getfileContent(path)
    
        
    if path.split('.')[-1] == 'kt':
        
        try:
            if filterOverlappingName(code.split('\n')[ int(line)-1 ], col, resourceId) == False:
                continue
        except IndexError: 
            failed += 1
            print('Index Error', resourceId, path, col, failed)
            
        counter += 1
        if(counter % 500 == 0):
            print("Index:", index, "parsed:", counter, "Failed:", failed)
        
        cols = row.tolist()
        cols.append(getResultForKotlin(code, int(line)))
        result.append( cols)
        


Index Error day /home/sajed/GitHub/apk-bakery/data crawler/code/Persian Calendar/PersianCalendar/src/main/kotlin/com/byagowi/persiancalendar/global/Global.kt 91 1
Index Error day /home/sajed/GitHub/apk-bakery/data crawler/code/Persian Calendar/PersianCalendar/src/main/kotlin/com/byagowi/persiancalendar/global/Global.kt 91 2
Index Error day /home/sajed/GitHub/apk-bakery/data crawler/code/Persian Calendar/PersianCalendar/src/main/kotlin/com/byagowi/persiancalendar/global/Global.kt 91 3
Index: 2215 parsed: 500 Failed: 3
Index: 2888 parsed: 1000 Failed: 3
Index: 3590 parsed: 1500 Failed: 3
Index: 4252 parsed: 2000 Failed: 3
Index: 4955 parsed: 2500 Failed: 3
Index: 5622 parsed: 3000 Failed: 3
Index: 6296 parsed: 3500 Failed: 3
Index: 7006 parsed: 4000 Failed: 3
Index: 7668 parsed: 4500 Failed: 3
Index: 8363 parsed: 5000 Failed: 3
Index: 9048 parsed: 5500 Failed: 3
Index: 9724 parsed: 6000 Failed: 3
Index: 10426 parsed: 6500 Failed: 3
Index: 11029 parsed: 7000 Failed: 3
Index: 11640 parsed:

In [11]:

kt = pd.DataFrame(result)

,0,1,2,3,4,5,6
0,com.byagowi.persiancalendar,crashScope/com.byagowi.persiancalendar-7.5.0/x...,['crashScope/com.byagowi.persiancalendar-7.5.0...,com.byagowi.persiancalendar:id/previous,/home/sajed/GitHub/apk-bakery/data crawler/cod...,"LocationRange(start=Location(line=95, col=25, ...",init {\n binding.monthView.initialize(share...
1,com.byagowi.persiancalendar,crashScope/com.byagowi.persiancalendar-7.5.0/x...,['crashScope/com.byagowi.persiancalendar-7.5.0...,com.byagowi.persiancalendar:id/next,/home/sajed/GitHub/apk-bakery/data crawler/cod...,"LocationRange(start=Location(line=109, col=25,...",init {\n binding.monthView.initialize(share...
2,com.byagowi.persiancalendar,crashScope/com.byagowi.persiancalendar-7.5.0/x...,['crashScope/com.byagowi.persiancalendar-7.5.0...,com.byagowi.persiancalendar:id/next,/home/sajed/GitHub/apk-bakery/data crawler/cod...,"LocationRange(start=Location(line=18, col=67, ...",
3,com.byagowi.persiancalendar,crashScope/com.byagowi.persiancalendar-7.5.0/x...,['crashScope/com.byagowi.persiancalendar-7.5.0...,com.byagowi.persiancalendar:id/next,/home/sajed/GitHub/apk-bakery/data crawler/cod...,"LocationRange(start=Location(line=30, col=48, ...",init {\n var k = floor((dayToYear(date) - 1...
4,com.byagowi.persiancalendar,crashScope/com.byagowi.persiancalendar-7.5.0/x...,['crashScope/com.byagowi.persiancalendar-7.5.0...,com.byagowi.persiancalendar:id/next,/home/sajed/GitHub/apk-bakery/data crawler/cod...,"LocationRange(start=Location(line=54, col=26, ...",init {\n var k = floor((dayToYear(date) - 1...
...,...,...,...,...,...,...,...
257538,com.fsck.k9,crashScope/com.fsck.k9-5.807/xmls/com.fsck.k9-...,['crashScope/com.fsck.k9-5.807/screenshots/com...,com.fsck.k9:id/next,/home/sajed/GitHub/apk-bakery/data crawler/cod...,"LocationRange(start=Location(line=99, col=29, ...","@Throws(XmlPullParserException::class, IOExcep..."
257539,com.fsck.k9,crashScope/com.fsck.k9-5.807/xmls/com.fsck.k9-...,['crashScope/com.fsck.k9-5.807/screenshots/com...,com.fsck.k9:id/next,/home/sajed/GitHub/apk-bakery/data crawler/cod...,"LocationRange(start=Location(line=1112, col=21...",private fun configureMenu(menu: Menu?) {\n ...
257540,com.fsck.k9,crashScope/com.fsck.k9-5.807/xmls/com.fsck.k9-...,['crashScope/com.fsck.k9-5.807/screenshots/com...,com.fsck.k9:id/next,/home/sajed/GitHub/apk-bakery/data crawler/cod...,"LocationRange(start=Location(line=1113, col=17...",private fun configureMenu(menu: Menu?) {\n ...
257541,com.fsck.k9,crashScope/com.fsck.k9-5.807/xmls/com.fsck.k9-...,['crashScope/com.fsck.k9-5.807/screenshots/com...,com.fsck.k9:id/next,/home/sajed/GitHub/apk-bakery/data crawler/cod...,"LocationRange(start=Location(line=1114, col=17...",private fun configureMenu(menu: Menu?) {\n ...


In [12]:
kt.to_excel( 'kotlin-mapping-test.xlsx', sheet_name='data', index = False)

In [13]:
len(kt)

257543

In [14]:
kt[0].unique()

array(['com.byagowi.persiancalendar', 'com.iven.musicplayergo',
       'com.greenaddress.greenbits_android_wallet', 'ws.xsoh.etar',
       'com.duckduckgo.mobile.android', 'com.better.alarm', 'com.fsck.k9'],
      dtype=object)